# How to stream events from within a tool

If your LangGraph graph needs to use tools that call LLMs (or any other LangChain `Runnable` objects -- other graphs, LCEL chains, retrievers, etc.), you might want to stream events from the underlying `Runnable`. This guide shows how you can do that.

## Setup

```bash
npm install @langchain/langgraph @langchain/anthropic zod
```

```typescript
process.env.ANTHROPIC_API_KEY = 'YOUR_API_KEY'
```

## Define graph and tools

We'll use a prebuilt ReAct agent for this guide

In [15]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatAnthropic } from "@langchain/anthropic";

const model = new ChatAnthropic({
  model: "claude-3-5-sonnet-20240620",
  temperature: 0,
})

const getItems = tool(
  async (input, config) => {
    const template = ChatPromptTemplate.fromMessages([
      [
        "human",
        "Can you tell me what kind of items i might find in the following place: '{place}'. " +
        "List at least 3 such items separating them by a comma. And include a brief description of each item..",
      ]
    ]);

    const modelWithConfig = model.withConfig({
      runName: "Get Items LLM",
      tags: ["tool_llm"],
    });

    const chain = template.pipe(modelWithConfig);
    const result = await chain.invoke(input, config);
    return result.content;
  },
  {
    name: "get_items",
    description: "Use this tool to look up which items are in the given place.",
    schema: z.object({
      place: z.string(),
    }),
  }
)

We're adding a custom tag (`tool_llm`) to our LLM runnable within the tool. This will allow us to filter events that we'll stream from the compiled graph (`agent`) Runnable below

In [18]:
import { createReactAgent } from "@langchain/langgraph/prebuilt";

const agent = createReactAgent({
  llm: model,
  tools: [getItems],
})


## Stream events from the graph

In [ ]:
let finalEvent
for await (const event of agent.streamEvents(
  {
    messages: [["human", "what items are on the shelf?"]],
  },
  {
    version: "v2"
  },
  {
    includeTags: ["tool_llm"],
  }
)) {
  console.log(event.data);
  finalEvent = event
}

Let's inspect the last event to get the final list of messages from the agent

In [7]:
const finalMessages = finalEvent.data.output.messages;
console.dir(finalMessages.map((msg) => ({
  type: msg._getType(),
  content: msg.content,
  tool_calls: msg.tool_calls,
})), { depth: null });

You can see that the content of the `ToolMessage` is the same as the output we streamed above